In [1]:
#Example XGBRegression
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import scipy
from sklearn.model_selection import GroupKFold
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from lineartree import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import metrics
import matplotlib.pyplot as plt
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

import pandas as pd
import numpy as np
import keras
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics

In [2]:
from numpy.random import seed
seed(1)
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [3]:
def rae(actual, predicted):
    actual = np.asarray(actual).reshape(-1,1)
    predicted = np.asarray(predicted).reshape(-1,1)
    numerator = np.sum(np.abs(predicted - actual))
    denominator = np.sum(np.abs(np.mean(actual) - actual))
    return numerator / denominator

In [4]:
def peak_eV_nm(peak_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    return 1239.84193/peak_eV

In [5]:
def FWHM_eV_nm(peak_eV,fwhm_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    fwhm_eV = np.asarray(fwhm_eV).reshape(-1,1)
    return  2*(1239.84193/(peak_eV-fwhm_eV/2) - 1239.84193/(peak_eV))

In [6]:
def NN(space):    
    #Test Linear Regression on Train/Val set, 10-fold cross validation (Group splits), UV Peak, FWHM (eV), Normalized

    # define model (with dropout)
    initializer = keras.initializers.HeNormal(seed=1)

    regressor_peak = keras.models.Sequential()
    regressor_peak.add(Dense(6, input_shape=(6,), kernel_initializer=initializer, activation='relu'))
    regressor_peak.add(Dense(50,  activation='relu'))
    regressor_peak.add(keras.layers.Dropout(space['x1']))
    regressor_peak.add(Dense(50,  activation='relu'))
    regressor_peak.add(keras.layers.Dropout(space['x1']))
    regressor_peak.add(Dense(1, kernel_initializer=initializer))

    # Compile model

    opt = keras.optimizers.Adam(learning_rate=space['x3'])
    regressor_peak.compile(loss='mean_squared_error', optimizer= opt)

    # load dataset into Pandas DataFrame (cleaned UV data)
    df = pd.read_csv('CdSe UV Train Val.csv')

    #normalize input and output data
    min_params = pd.Series([0, 0, 3, 200, 0, 400, 15, 1239.84193/620, 0.05], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
    max_params = pd.Series([50, 50, 150, 310, 600, 620, 80, 1239.84193/400, 0.4], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
    df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] = (df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] - min_params) / (max_params- min_params)


    X = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s', 'Peak_eV']]
    y = df['FWHM_eV']

    from sklearn.model_selection import GroupShuffleSplit 

    splitter = GroupShuffleSplit(test_size=.1, n_splits=2, random_state = 1)
    split = splitter.split(df, groups=df['Expt. No'])
    train_inds, test_inds = next(split)

    train = df.iloc[train_inds]
    test = df.iloc[test_inds]

    X_train = train[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s', 'Peak_eV']]
    y_train = train['FWHM_eV']

    X_test = test[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s', 'Peak_eV']]
    y_test = test['FWHM_eV']

    X_train = K.constant(X_train)
    y_train = K.constant(y_train)
    X_test = K.constant(X_test)
    y_train = K.constant(y_train)
            
    matrix = np.zeros([10,13])
    i = 0
    j = 0
        
    #reset weights of model
    regressor_peak.load_weights('initialize_weights4.h5')
    
    #prediction for Peak
    regressor_peak.fit(X_train, y_train, epochs=int(space['x2']), validation_data=(X_test, y_test),verbose = 0, use_multiprocessing=False, workers = 100)
    y_pred = regressor_peak.predict(X_test)

    #errors in eV
    matrix[10*i + j,0] = metrics.mean_absolute_error(y_test, y_pred)
    matrix[10*i + j,1] = metrics.r2_score(y_test, y_pred)
    matrix[10*i + j,2] = rae(y_test, y_pred)
    matrix[10*i + j,3] = np.sqrt(metrics.mean_squared_error(y_test, y_pred))

    #errors in nm
    #UNnormalize the data first
    y_test = y_test*(1239.84193/420 - 1239.84193/620)+1239.84193/620
    y_pred = y_pred*(1239.84193/420 - 1239.84193/620)+1239.84193/620
    matrix[10*i + j,4] = metrics.mean_absolute_error(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[10*i + j,5] = metrics.r2_score(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[10*i + j,6] = rae(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[10*i + j,7] = np.sqrt(metrics.mean_squared_error(peak_eV_nm(y_test), peak_eV_nm(y_pred)))
    j+=1

    matrix_2 = np.zeros([1,21])
    matrix_2[0,:8] = 10*matrix.mean(0)[:8]

    print('\033[1m' + 'Absorption Peak' + '\033[0m')
    print("MAE (norm, eV):", matrix_2[0,0])
    print("R2 (norm, eV):", matrix_2[0,1])
    print("RAE (norm, eV):", matrix_2[0,2])
    print("RMSE (norm, eV):", matrix_2[0,3])
    print("MAE (nm):", matrix_2[0,4])
    print("R2 (nm):", matrix_2[0,5])
    print("RAE (nm):", matrix_2[0,6])
    print("RMSE (nm):", matrix_2[0,7])

    return {'loss': matrix_2[0,3], 'status': STATUS_OK }

In [7]:
rstate = np.random.default_rng(42)

from hyperopt import fmin, tpe, hp, STATUS_OK

space = {'x1': hp.uniform('x1', 0.1,0.5),
         'x2': hp.quniform('x2', 10,600,20),
         'x3': hp.uniform('x3', 0.0001,0.01)
        }

best = fmin(NN,
            space=space,
            algo=tpe.suggest,
            max_evals=200,
            rstate=rstate)

print(best)

4/4 [==============================] - 0s 3ms/step                                                                    

Absorption Peak                                                                                                
MAE (norm, eV):                                                                                                        
0.029936081643104556                                                                                                   
R2 (norm, eV):                                                                                                         
0.2922106358645976                                                                                                     
RAE (norm, eV):                                                                                                        
0.5717831324592345                                                                                                     
RMSE (norm, eV):                                

MAE (nm):                                                                                                              
6.985244410215029                                                                                                      
R2 (nm):                                                                                                               
0.4240734086878707                                                                                                     
RAE (nm):                                                                                                              
0.6066896953335281                                                                                                     
RMSE (nm):                                                                                                             
12.851432571983377                                                                                                     
4/4 [==============================] - 0

MAE (norm, eV):                                                                                                        
0.03142403470720563                                                                                                    
R2 (norm, eV):                                                                                                         
0.19049341254725294                                                                                                    
RAE (norm, eV):                                                                                                        
0.6002032334626668                                                                                                     
RMSE (norm, eV):                                                                                                       
0.07349868083390848                                                                                                    
MAE (nm):                               

0.1018277665831252                                                                                                     
RAE (nm):                                                                                                              
0.6630858891404698                                                                                                     
RMSE (nm):                                                                                                             
16.048993047839186                                                                                                     
4/4 [==============================] - 0s 3ms/step                                                                    

Absorption Peak                                                                                                
MAE (norm, eV):                                                                                                        
0.043571322755132424                            

0.15945530863882273                                                                                                    
RAE (norm, eV):                                                                                                        
0.6121004943858981                                                                                                     
RMSE (norm, eV):                                                                                                       
0.07489447065220574                                                                                                    
MAE (nm):                                                                                                              
7.0191248838754285                                                                                                     
R2 (nm):                                                                                                               
0.21038808141071486                     

...

0.06247508324356345                                                                                                    
MAE (nm):                                                                                                              
8.747868794250579                                                                                                      
R2 (nm):                                                                                                               
0.4383563807695362                                                                                                     
RAE (nm):                                                                                                              
0.7597789772166594                                                                                                     
RMSE (nm):                                                                                                             
12.691074428686399                      

Absorption Peak                                                                                                
MAE (norm, eV):                                                                                                        
0.029887852638789583                                                                                                   
R2 (norm, eV):                                                                                                         
0.233560709593003                                                                                                      
RAE (norm, eV):                                                                                                        
0.5708619520759308                                                                                                     
RMSE (norm, eV):                                                                                                       
0.07151682585101303                             

R2 (nm):                                                                                                               
0.44675897662514474                                                                                                    
RAE (nm):                                                                                                              
0.5951595377294664                                                                                                     
RMSE (nm):                                                                                                             
12.595782839169072                                                                                                     
4/4 [==============================] - 0s 0s/step                                                                     

Absorption Peak                                                                                                
MAE (norm, eV):                                 

R2 (norm, eV):                                                                                                         
0.1509888121130064                                                                                                     
RAE (norm, eV):                                                                                                        
0.7270551795885696                                                                                                     
RMSE (norm, eV):                                                                                                       
0.0752707177604078                                                                                                     
MAE (nm):                                                                                                              
8.284416591610244                                                                                                      
R2 (nm):                                

0.5926201265247479                                                                                                     
RMSE (nm):                                                                                                             
14.090790884229367                                                                                                     
4/4 [==============================] - 0s 0s/step                                                                     

Absorption Peak                                                                                                
MAE (norm, eV):                                                                                                        
0.02805587344782694                                                                                                    
R2 (norm, eV):                                                                                                         
0.6105512448382708                              